In [1]:
import os

In [2]:
%pwd

'/workspaces/project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/workspaces/project'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/jcole313/project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="jcole313"
os.environ["MLFLOW_TRACKING_PASSWORD"]="71368345455e3aa4cabbe321bbd6c4405a0db448"

In [6]:
import tensorflow as tf

2024-04-03 08:06:48.315913: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-03 08:06:50.614052: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-03 08:06:50.614097: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-03 08:06:50.631664: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-03 08:06:51.702868: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-03 08:06:51.704230: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    testing_data:Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [14]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data/Data/train",
            testing_data="artifacts/data_ingestion/Chest-CT-Scan-data/Data/test",
            mlflow_uri="https://dagshub.com/jcole313/project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [15]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from tensorflow.keras.applications import resnet

In [16]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _test_generator(self):

        datagenerator_kwargs = dict(
            dtype='float32',
            preprocessing_function=resnet.preprocess_input
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            class_mode = 'categorical'
        )

        test_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.test_generator = test_datagenerator.flow_from_directory(
            directory=self.config.testing_data,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._test_generator()
        self.score = model.evaluate(self.test_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="ResNet50Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [17]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-04-03 08:45:00,500: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-03 08:45:00,529: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-03 08:45:00,530: INFO: common: created directory at: artifacts]


Found 315 images belonging to 4 classes.


2024-04-03 08:45:03.671013: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 216678400 exceeds 10% of free system memory.
2024-04-03 08:45:04.228245: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 216678400 exceeds 10% of free system memory.
2024-04-03 08:45:04.282374: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 220463104 exceeds 10% of free system memory.
2024-04-03 08:45:04.323993: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 216678400 exceeds 10% of free system memory.
2024-04-03 08:45:04.392568: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 216678400 exceeds 10% of free system memory.


20/20 [==============================] - 75s 4s/step - loss: 0.6522 - acc: 0.7397
[2024-04-03 08:46:17,955: INFO: common: json file saved at: scores.json]


2024/04/03 08:46:19 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /tmp/tmpc0gpt1o2/model/data/model/assets
[2024-04-03 08:46:31,305: INFO: builder_impl: Assets written to: /tmp/tmpc0gpt1o2/model/data/model/assets]


/workspaces/project/env/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'ResNet50Model'.
2024/04/03 08:47:19 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ResNet50Model, version 1
Created version '1' of model 'ResNet50Model'.
